# 本单元用于记录LeetCode每日一题

## 问题1：[HTML 实体解析器](https://leetcode.cn/problems/html-entity-parser/description/)
### 题目
字符串替换, 将字符串中的特殊符号, 根据规则转化为指定符号。
### 分析
- 使用正则表达式进行划分, 由于我们发现待替换的字符串中都是“&xxx;”的形式，因此我们使用正则表达式进行分组
- re.split(r"(&[^&;]+;)", text):
    - [^&;]: 表示匹配不包括&和;符号
    - +: 表示前面的匹配至少出现一次
    - [^&;]+: 表示必须匹配一个非&和非;的字符
    - (): 表示将内部匹配到的东西作为一个分组整体返回(保证了分割字符中包含有&;) 
### 优化
- 直接使用replace(目标值, 更换值)进行替换即可


In [ ]:
import re
def process(text):
    word_dict = {"&quot;":"\"","&apos;":"\'","&amp;":"&", "&gt;":">","&lt;":"<","&frasl;":"/"}
    word_list = re.split(r"(&[^&;]+;)", text)
    print(word_list)
    for index, word in enumerate(word_list):
        if word in word_dict:
            word_list[index] = word_dict[word]
    return "".join(word_list)

text = "and I quote: &quot;...&quot;"
process(text)


## 问题2：[确定两个字符串是否接近](https://leetcode.cn/problems/determine-if-two-strings-are-close/description/)
### 题目
如果可以使用以下操作从一个字符串得到另一个字符串，则认为两个字符串接近:
- 交换任意两个现有字符的位置.
- 将某种字符与另一种字符全部互换。
若通过有限步的上述两种操作，可以从原字符串得到另一个字符串则认为两个字符串相近

### 分析
- 由于可以交换任意两个字符位置，因此只要保证原字符串与目标字符串的每种字符个数是一样的即可。
- 又由于可以交换任意两种字符(即把所有的a变成b,同时把所有的b变成a), 因此只用保证原字符串的中字符个数列表与目标字符串字符个数列表具有相同元素即可.(例如[1,2,1] 与 [2,1,1])

### 注意
- 由于两个步骤必须与现有的字符交换，因此如果目标字符串中有字母a，但原字符串中没有字母a那将永远无法转换。

In [ ]:
from collections import Counter

def process(word1, word2):
    ## 得到每个字母计数的数值列表
    count1 = Counter(word1)
    count2 = Counter(word2)

    if set(count1.keys()) != set(count2.keys()):
        return False
    
    count1_list = count1.values()
    count2_list = count2.values()

    ### 判断word的元素个数列表是否相同
    count = {}
    for x in count1_list:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    for x in count2_list:
        if x not in count or count[x] == 0:
            return False
        count[x] -= 1

    # 以上代码可以通过以下方式替换(时间复杂度会上升)
    # if sorted(count1_list) != sorted(count2_list):
    #     return False

    return True

word1 = "uau"
word2 = "ssx"
process(word1, word2)


## 问题3：[最小体力消耗路径(并查集)](https://leetcode.cn/problems/path-with-minimum-effort)
### 题目
一开始你在最左上角的格子 (0, 0) ，且你希望去最右下角的格子 (rows-1, columns-1) （注意下标从 0 开始编号）。你每次可以往 上，下，左，右 四个方向之一移动，你想要找到耗费 体力 最小的一条路径。一条路径耗费的 体力值 是路径上相邻格子之间 高度差绝对值 的 最大值 决定的。

请你返回从左上角走到右下角的最小 体力消耗值 。

### 分析
- 如果使动态规划dp[i][j]表示到i,j位置的最小体力, 先从左到右遍历更新上,左位置，再用右到左遍历更新下，右位置。但实际上这种方式是不对的。因为方向的改变会导致对前面的状态改变，因此不可取。
- 如果把每个一个格子看作是一个地点，那这个题目则是让我们找最短路径，只不过这个最短路径的定义不同。我们可以使用并查集方式，每次放入一个点，当我们加入一条权值为 x 的边之后，如果左上角和右下角从非连通状态变为连通状态，那么 x即为答案(我们先根据权重从小到大排列)。

### 注意
- 联通性判断类型的题目都可以使用并查集


In [ ]:
class UnionFind:
    """"标准并查集对象
    """
    def __init__(self, n: int):
        self.pre = list(range(n))  ## 初始化每一个点都是以自己为主元

    ## 查找函数
    def find(self, x):
        if self.pre[x] == x:
            return x
        else:
            self.pre[x] = self.find(self.pre[x])
            return self.pre[x]

    ## 并函数
    def join(self, x, y):
        fx = self.find(x)
        fy = self.find(y)
        if fx != fy:
            self.pre[fx] = fy # 定义fx的上级为fy

    ## 连通性判断x,y是否联通
    def connected(self, x: int, y: int) -> bool:
        x, y = self.find(x), self.find(y)
        return x == y


def process(heights):
    m, n = len(heights), len(heights[0])
    edges = []  # 构造边对象
    for i in range(m):
        for j in range(n):
            iden = i * n + j # 当前的坐标拉平后的位置
            if i > 0:  ## 由于是双向关系，因此只用保证上下，左右关系即可
                edges.append((iden - n, iden, abs(heights[i][j] - heights[i - 1][j])))
            if j > 0:
                edges.append((iden - 1, iden, abs(heights[i][j] - heights[i][j - 1])))
    edges = sorted(edges, key=lambda x: x[2])
    uf = UnionFind(m * n)  # 初始化并查集
    ans = 0

    for x, y, v in edges:
        uf.join(x, y)
        if uf.connected(0, m * n - 1):
            ans = v
            break
    return ans

## 问题4：[猜数字游戏](https://leetcode.cn/problems/bulls-and-cows/)
### 题目
给你一个秘密数字 secret 和朋友猜测的数字 guess ，请你返回对朋友这次猜测的提示。提示的格式为 "xAyB" ，x 是公牛个数， y 是奶牛个数，A 表示公牛，B 表示奶牛。请注意秘密数字和朋友猜测的数字都可能含有重复数字。
- 猜测数字中有多少位属于数字和确切位置都猜对了（称为 "Bulls"，公牛），
- 有多少位属于数字猜对了但是位置不对（称为 "Cows"，奶牛）。也就是说，这次猜测中有多少位非公牛数字可以通过重新排列转换成公牛数字。

### 分析
- 维护两个数组true_num和guess_num, true_num[i]表示数字i在secret中出现的次数，guess_num[i]表示数字i在guess中出现的次数
- 同时遍历secret 和 guess的i位置, 如果secret[i] == guess[i]则公牛数字加1, 否则
    - 先进行判断, 当前的正确数字是否已经出现在了guess_num中, 如果是则奶牛数字加1, 同时guess_num对应位置数字出现次数减一
    - 当前的正确数字还没出现在guess_num中，则true_num对应位置+1
    - 再判断猜测数字，是否已经出现在了true_num中, 如果是则奶牛数字加1, 同时true_num对应位置数字出现次数减一
    - 当前的猜测数字还没出现在true_num中，则guess_num对应位置+1


In [ ]:
def process(secret, guess):
    true_nums = [0 for _ in range(10)]
    guess_nums = [0 for _ in range(10)]
    bulls = 0   # 公牛结果
    cows = 0    # 奶牛结果
    for true_num, guess_num in  zip(secret, guess):
        # 判断是否为公牛数字
        if true_num == guess_num:
            bulls += 1
            continue
        
        # 判断是否为奶牛数字
        if guess_nums[int(true_num)] != 0:
            cows += 1
            guess_nums[int(true_num)] -= 1
        else:
            true_nums[int(true_num)] += 1
        
        # 继续判断是否为奶牛数字
        if true_nums[int(guess_num)] != 0:
            cows += 1
            true_nums[int(guess_num)] -= 1
        else:
            guess_nums[int(guess_num)] += 1
        
    return f"{bulls}A{cows}B"

secret = "1123"
guess = "0111"
process(secret, guess)

## 问题5：[需要添加的硬币的最小数量](https://leetcode.cn/problems/minimum-number-of-coins-to-be-added/description/)
### 题目
给你整数数组coins, 表示可用的硬币的面值, 以及一个整数target。如果存在某个coins的子序列总和为x，那么整数x就是一个可取得的金额。我们需要往coins中添加数字, 使得对于[1, target]的每一个值都是可取得的(即,都等于coins中某一个子序列的总和)。请返回最少需要添加多少个数字。

### 分析
- 我们需要将coins进行排序后遍历, 从最小值开始, 如果我们可以得到[1, x]中任意数都可以取得。
    - 此时如果下一个硬币比x大且不是x+1这个数值, 那我们将永远无法得到x+1(因为后面的coin都比x+1大, 意味着这个x+1是无法被取得), 因此我们必须添加x+1这个硬币。当我们添加了x+1这个硬币, 此时我们可以得到的取值范围就变成了[x+1, x+x+1], 且[1, x]我们依然可以取得到, 因此添加完x+1这个硬币后, 我们可以取得的范围为[1, x+x+1]
    - 如果这个值比x小(假设为s), 意味着我们可以不用添加硬币, 而还可以扩大取值范围依到[1, x+s]
- 一直循环直到我们的范围覆盖了[1, target]
- 有可能我们循环了完了coins都没能达到target, 我们还得继续添加, 因此使用了一个技巧, 在while中使用index来控制是否前进数组。如果我们添加硬币, 则还需要判定当前的coin, 因此我们不用增加index, 从而实现重复判定。


In [ ]:
def process(coins, target):
    coins.sort()
    ans = 0
    max_coin = 0  # 初始为0
    index = 0
    while max_coin < target:
        if index < len(coins) and coins[index] <= max_coin+1:   # 如果小于等于最大范围+1, 我们都可以不用添加硬币, 从而增大范围
            max_coin += coins[index]
            index += 1
        else:          # 如果当前硬币大于了最大范围, 我们需要添加一个max_coin+1元素
            ans += 1  
            max_coin = max_coin*2 + 1   # 注意我们此时并不增加index, 继续让其判定当前的coin

    return ans

coins = [1,4,10]
target = 19
process(coins, target)

## 问题6：[访问完所有房间的第一天](https://leetcode.cn/problems/first-day-where-you-have-been-in-all-the-rooms/description/)
### 题目
给你一个长度为n数组nextVisit(访问房间号)。在接下来的几天中，你访问房间的次序将根据下面的规则决定：假设某一天，你访问i号房间。
- 如果算上本次访问，访问 i 号房间的次数为**奇数**，那么第二天需要访问第nextVisit[i]号房间 (数据会保证 0<= nextVisit[i] <= i)。
- 如果算上本次访问，访问 i 号房间的次数为**偶数**，那么第二需要访问(i + 1) mod n 号房间 (访问下一间, 如果i为最后一间, 则从头访问第0间)。

请返回你访问完所有房间日期编号(也就是多少天后, 你第一次把所有的房间都访问完).

### 分析
- 注意到关键点 nextVisit[i] <= i, 意味着当奇数次访问第i号房间时, 一定会回访前面的房间。因此当我访问到第i号房间时, 对于i左边的房间, 我们一定都访问了偶数次(不然不可能到达i, 可以反证)。
- 当我们从i号房间需要回访到j号房间时, 此时[j, i−1]范围内的房间都处于访问偶数次的状态。那么当我们访问这个范围内的每个房间时，算上本次访问，访问次数一定是奇数。所以要想重新回到i, 对于[j, i−1]范围内的每个房间，我们都需要执行一次「回访」。
- f[i] 表示从 「访问到房间i且次数为奇数」 到 「访问到房间i且次数为偶数」 所需要的天数。这个天数是固定的, 只有经历过这个天数后, 才能访问到下一间i+1号房间。注意: 首次访问房间i的一天，和重新回到房间i的一天, 因此状态转移时+2天
- 状态转移方程:f[i] = 2 + sum(f[j], f[2]...f[i-1]), 初始状态f[0]=0, 这里的j为nextVisit[i]
- 我们最后的答案是: sum(f[0], f[1], ..., f[n-1]) + 1; 这里加1是访问第n间的那天
- 由于我们这里需要多次求和, 因此我们定义f[n]的前n-1项和（前缀和）s[n] = sum(f[0], f[1], ..., f[n-1]), 状态转移方程就变为了: f[n] = s[n] - s[n-1] + 2。 我们的最终结果即求: s[n] + 1
- 最后的转移方程为: s[i+1] = s[i] * 2 - s[j] + 2

In [ ]:
def process(nextVisit):
    s = [0] * len(nextVisit)
    for index, num in enumerate(nextVisit[:-1]):
        s[index+1] = s[index] * 2 - s[num] + 2
    return s[-1]+1-1    # 我们求的是日期, 我们从第0天开始的(第0天的访问算天数的), s[-1]+1是经过了多少天

nextVisit = [0,0]
process(nextVisit)

## 问题7：[验证二叉树的前序序列化](https://leetcode.cn/problems/verify-preorder-serialization-of-a-binary-tree/description/)
### 题目
给定一串以逗号分隔的序列("9,3,4,#,#,1,#,#,2,#,6,#,#")，验证它是否是正确的二叉树的**前序序列化**。编写一个在不重构树的条件下的可行算法。注意不允许重建树

### 分析
- 我们知道在树（甚至图）中，所有节点的入度(该节点有几个父节点)之和等于出度(该节点有几个子节点)之和。
- 每个非#节点, 会提供总入度+1, 总出度+2; 每个#节点, 会提供总入度+1, 总出度+0; 注意根节点只提供2个出度
- 遍历整个序列, 如果总入度和总出度的差小于0则说明该二叉数无效。注意在遍历中如果再加入某个节点前已经出现了入度 == 出度, 说明不可能再能加入新的节点(因为入度==出度意味着每个叶节点都是null了)

### 方案二: 模拟递归栈
- 把有效的叶子节点使用 "#" 代替。 比如把 4## 替换成 # 。此时，叶子节点会变成空节点！
- 先把节点压入栈中, 检查栈顶元素是否满(数字 # #)的结构, 如果时则抛出栈顶三元素, 把#压入栈顶(即把 4## 替换成 #), 循环上述操作, 直到无法压缩, 再遍历下一个节点。
- 最后判断是否只剩下["#"]即可

In [ ]:
def process(preorder):
    nodes = preorder.split(",")

    if nodes[0] == "#" and len(nodes) > 1:  # 根节点为空, 直接返回
        return False

    rudu = -1    # 根节点本是不增加入度的
    chudu = 0    
    for node in nodes:
        if rudu == chudu:   # 如果入度等于出度, 说明在进入这个节点之前, 已经达到了是完整二叉树了, 不能再加节点了
            return False
        
        rudu += 1           # 无论是否为空节点, 都会增加入度
        if node != "#":
            chudu += 2
        
    return chudu-rudu == 0

preorder = "7,2,#,2,#,#,#,6,#"
process(preorder)

In [ ]:
# 方案二使用栈
def process(preorder):
    stack = []
    for node in preorder.split(','):
        stack.append(node)
        while len(stack) >= 3 and stack[-1] == stack[-2] == '#' and stack[-3] != '#':
            stack.pop(), stack.pop(), stack.pop()
            stack.append('#')
    return len(stack) == 1 and stack.pop() == '#'


## 问题8：[统计将重叠区间合并成组的方案数](https://leetcode.cn/problems/count-ways-to-group-overlapping-ranges/description/)
### 题目
给你一个二维整数数组ranges，其中ranges[i] = [start, end]表示 start 到 end之间(包括两端点)的所有整数都包含在第i个区间中。请返回将ranges分为两个组, 满足以下规则: 
- 如果ranges[i]与ranges[j]有重叠, 则两个区间必须在一个组内。
- 每个ranges[i]只能在一个组中。
- 组可以没有元素

请返回 组的方案数(有多少中划分)

### 分析
- 我们可以将区间进行整合，看能得到多少个互不重叠的整合区间。先将区间排序, 并遍历每个区间, 如果当前区间和**整合区间**有重叠, 则更新整合区间左右边界(只用更新右边界), 如果没有重叠, 则说明当前区间为下一个整合区间的起始区间。
- 遍历结束后, 返回整和区间个数, 分为两组的情况数为(2**整合区间个数)


In [ ]:
def process(ranges):
    ranges.sort(key=lambda x: x[0])
    left = ranges[0][0]
    right = ranges[0][1]
    count = 1
    for x, y in ranges:
        print(x,y)
        if left <= x <= right:
            right = max(right, y)  # 合并区间
        else:
            count += 1  # 整合区间数 +1
            left = x    # 重置整合区间的左右边界
            right = y
    return pow(2, count, 1_000_000_007)

ranges = [[6,10],[5,15]] 
process(ranges)

## 问题9：[所有可能的真二叉树](https://leetcode.cn/problems/all-possible-full-binary-trees/description/)
### 题目
给你一个整数n, 请你找出所有可能含n个节点的真二叉树(树中每个节点的值为0). 返回的结果是每种情况的根节点(即List[Node])。

真二叉树定义:树中每个节点恰好有 0 或 2 个子节点

### 分析
- 每个结点的子结点数是0或2，此时可以推出真二叉树中的结点数n为奇数(由归纳法证明,每次加节点只能2个2个加), 因此当n为偶数时直接返回null
- 真二叉树的左右子树也一定是真二叉树, 因此我们可以采用分治法。对于节点为n的真二叉树的所有情况我们是可以得到的, 因此只要我们知道左右树的节点个数, 即可组合得到所有可能的真二叉树。假设左子树的数目为i, 则右子树的节点数目则为 n−1-i (因为左右子数结点和为n-1)，则我们可以推出左子树与右子树的节点数目序列为：$$[(1,n−2),(3,n−4),(5,n−6),⋯,(n−2,1)]$$
- 当 n=1时，此时只有一个结点的二叉树是真二叉树；当 n>1时，分别枚举左子树和右子树的根结点数，然后递归地构造左子树和右子树，并返回左子树与右子树的根节点列表。确定左子树与右子树的根节点列表后，分别枚举不同的左子树的根节点与右子树的根节点，从而可以构造出真二叉树的根节点。


In [ ]:
from functools import cache
from typing import List, Optional


class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
@cache
def process(n) -> List[Optional[TreeNode]]:
    ans = []
    if n % 2 == 0:  # 如果为偶数则直接返回
        return ans
    if n == 1:  # 如果为1，则直接返回根节点
        ans.append(TreeNode(0))
        return ans
    # 分别递归左右子树, 左右子树的节点均为奇数
    for left_node in range(1, n-1, 2):
        left_ans = process(left_node)
        right_ans = process(n-1-left_node)
        # 将得到的左右子数的情况进行组合
        for left_node in left_ans:
            for right_node in right_ans:
                root = TreeNode(0, left_node, right_node)  # 这是一种组合
                ans.append(root)
    return ans
    


## 问题10：[使数组连续的最少操作数](https://leetcode.cn/problems/minimum-number-of-operations-to-make-array-continuous/description/)
### 题目
给你一个整数数组nums。每一次操作中，你可以将nums中任意一个元素替换成任意整数。请你返回使nums连续的最少操作次数。

数组连续定义:
- nums 中所有元素都是 互不相同 的。
- nums 中 最大 元素与 最小 元素的差等于 nums.length - 1 。

### 分析
- 正难则反, 我们可以计算出“最多有多少个元素不用改变”, 该问题其实就是在问固定长度的滑动空间(长度为nums.length - 1), 包含的最多点有多少个
- 我们首先将nums去重后排序(假设得到的数组为a), 使用滑动窗口模板, 先滑动右端点, 如果达到收缩条件(左右端点的差大于了nums.length - 1), 由于求最大，因此在收缩时记录答案


In [ ]:
def process(nums):
    n = len(nums)
    a = sorted(set(nums))
    ans = 0
    left = right = 0
    while right < len(a):  
        right += 1                    # 右边界右移
        # 此时窗口中的数值为[left, right)
        ans = max(ans, right-left)
        while right < len(a) and a[right] > a[left]+n-1: # 满足收缩条件
            left += 1
    
    return n-ans

nums = [1,2,3,10,20]
process(nums)

## 问题11：[树节点的第 K 个祖先](https://leetcode.cn/problems/kth-ancestor-of-a-tree-node/description/)
### 题目
给你一棵树，树上有n个节点，按从0到n-1编号。树以父节点数组的形式给出，其中parent[i]是节点i的父节点。树的根节点是编号为 0 的节点。树节点的第 k 个祖先节点是从该节点到根节点路径上的第 k 个节点。

实现 TreeAncestor 类:
- TreeAncestor(int n， int[] parent)对树和父数组中的节点数初始化对象。
- getKthAncestor(int node, int k) 返回节点 node 的第 k 个祖先节点。如果不存在这样的祖先节点，返回 -1 。

### 分析


## 问题12：[节点与其祖先之间的最大差值](https://leetcode.cn/problems/maximum-difference-between-node-and-ancestor/description/)
### 题目


### 分析

## 问题13：[有向无环图中一个节点的所有祖先](https://leetcode.cn/problems/all-ancestors-of-a-node-in-a-directed-acyclic-graph/description/)
### 题目
给你一个正整数n，它表示一个 有向无环图 中节点的数目，节点编号为[0, n-1]. 一个二维整数数组edges,其中edges[i] = [from, to] 表示图中一条从from到to的单向边(从节点from到节点to)。请你返回一个数组 answer，其中 answer[i]是第i个节点的所有祖先 ，这些祖先节点升序排序。

祖先定义: 如果 u 通过一系列边，能够到达 v ，那么我们称节点 u 是节点 v 的 祖先 节点。
### 分析
- 在遍历边时反向保存, key为目标节点, value为源节点集合, 最后得到的图为 {3:[1,2,4]} 表示3的父节节点为1,2,4
- 不断将源节点加入到目标节点的祖先集合中, 直到遍历结束(由于无环图, 因从总会遍历结束, 我们使用一个集合visited来保存遍历过的节点, 如果遍历过就跳过, 避免重复遍历)
- 我们使用while循环+队列的方式, 将每一个节点的父节点放入队列中, 这样可以遍历到父节点的父节点, 直到遍历完成

In [ ]:
import bisect
def process(n: int, edges):
    graph = [[] for _ in range(n)]
    for edge in edges:
        graph[edge[1]].append(edge[0])  # 构建图

    ans = [[] for _ in range(n)]

    for i in range(n):  # 遍历每个节点
        q = [i]         # 队列
        visited = [False] * n   # 保存遍历过的节点
        visited[i] = True
        while q:
            node = q.pop(0)
            for neighbor in graph[node]:
                if not visited[neighbor]:   # 如果这个父节点没有遍历过
                    visited[neighbor] = True
                    bisect.insort(ans[i], neighbor)  # 记录到答案中
                    q.append(neighbor)
    return ans

n = 8
edgeList = [[0,3],[0,4],[1,3],[2,4],[2,7],[3,5],[3,6],[3,7],[4,6]]
process(n, edgeList)

## 问题14：[修改后的最大二进制字符串](https://leetcode.cn/problems/maximum-binary-string-after-change/description)
### 题目
给你一个二进制字符串 binary ，它仅有 0 或者 1 组成。你可以使用下面的操作任意次对它进行修改：
- 操作 1: 如果二进制串包含子字符串 "00" ，你可以用 "10" 将其替换。比如， "00010" -> "10010"
- 操作 2: 如果二进制串包含子字符串 "10" ，你可以用 "01" 将其替换。比如， "00010" -> "00001"

请你返回执行上述操作任意次以后能得到的 最大二进制字符串。

### 分析
- 从操作一和操作二可以得出, 我们一直可以把0往左移动, 直到碰到00之后, 将其转化为10, 从而使得二进制字符增大。从这个角度来看, 最左边的0是无论无何都不可以被移动的, 而其他位置的0都可以向右移动, 与最左侧的0配合形成00->10的转换。
- 最终的最大二进制字符串就是除了最左边的0之外, 其他位置均为1(这种说法不对, 由于字符串可能以多个000开头, 此时都可以将0000->1110的形式), 因此连续的0开头的字符串, 我们应该找到最末尾处的那个0的位置
- 其实换个思路, 我们可以找到首个0的位置, 只经过步骤二即可将该位置后的所有1都压缩到尾部, 即为1111000000111111, 此时我们可以直到最终答案的那个0，就是在压缩尾部的1的左边
- 注意全为1的字符串和只有最后一位为0(最左侧的0刚好是最后一个位置)的字符串不用移动, 直接返回即可

### 技巧
- str.find(sub)返回子字符串sub在字符串str中第一次出现的位置, 如果没有找到, 返回-1
- str.count(sub, start, end)返回子字符串sub在字符串str中从start到end出现的次数, 如果没有找到, 返回0

In [ ]:
def process(binary):
    index_0 = binary.find('0')  # 找到最左侧的0的位置
    if index_0 == -1 or index_0 == len(binary) - 1 :
        return binary
    count_1 = binary.count('1', index_0)  # 统计首个0位置的右边有多少个1
    # 最终答案为
    ans = "1"*(len(binary)-1- count_1) + "0" + "1"*count_1

    return ans
    
process("01")

## 问题15：[互质树](https://leetcode.cn/problems/tree-of-coprimes/description/)
### 题目
给你一个n个节点的树（也就是一个无环连通无向图），节点编号从0到n-1，且恰好有n-1条边，每个节点有一个值。树的根节点为0号点。nums[i]表示第 i个节点的值(节点值可以重复), edges[j]=[u, v]表示节点u和节点v在树中有一条边。

无向图中祖先的定义: 从节点i到根最短路径上的点都是节点i的祖先节点。一个节点不是它自己的祖先节点。

请你返回一个大小为 n 的数组 ans ，其中 ans[i]是离节点i最近的祖先节点且满足nums[i]和nums[ans[i]]是互质的，如果不存在这样的祖先节点，ans[i]为-1。

### 分析
- 最暴力的做法是，枚举x的所有祖先节点(可以使用广度优先搜索), 从最近祖先开始, 直到找到和x互质的祖先. 但这种方式如果树刚好为一条链子,那么对于每个x,枚举其父节点复杂度是O(n), 因此所有节点遍历完就是O(n^2)
- 由于我们直到每个节点值介于[1,50]之间, 我们可以提前构造出一个映射, 用于保存在[1,50]范围内, 从1开始的每个数字的互质列表, coprime[i]表示与数值i互质的所有数(这些数值在1,50范围)
- 我们使用val_depth_id数组保存每个数值对应的节点信息，即val_depth_id[i]: 数值i为的节点的(层深度, 节点号), 由于存在数值重复, 我们需要使用回溯法, 恢复到上层同数值节点的信息。
- 在深度遍历时, 我们对每个节点, 都可以保存下他数值所对应的(层深度, 节点号), 每次更新val_depth_id[value]时, 都会使得该value对应的层数+1

In [ ]:
import math
# 互质匹配表
coprime = [[j for j in range(1, 51) if math.gcd(i, j) == 1]
           for i in range(51)]

def process(nums, edges):
    n = len(nums)
    ans = [-1] * n
    graph = [[] for _ in range(n)]  # 构造树
    for x, y in edges:
        graph[x].append(y)
        graph[y].append(x)
    
    val_depth_id = [(-1, -1)] * 51  # 保存数值对应的节点信息

    # 深度优先遍历, 当前节点为x, 父节点为parent, x的节点深度为depth
    def dfs(x, parent, depth):
        value = nums[x]  # 获得当前节点的值

        # 根据节点值, 找到与节点值互质的数值, 并找到这些数值对应的节点信息
        # 这里通过max函数找到最大深度, 也就是最临近的
        ans[x] = max([val_depth_id[j] for j in coprime[value]])[1]

        # 保存回溯值
        tmp = val_depth_id[value]
        # 更新val_depth_id, 将当前节点的层数和节点号进行更新
        val_depth_id[value] = (depth, x)
        
        for child in graph[x]:
            if child == parent:  # 跳过父节点
                continue
            dfs(child, x, depth+1)
        
        # 回溯上层的val_depth_id状态
        val_depth_id[value] = tmp
    
    dfs(0, -1, 0)  # 从根节点开始遍历
    return ans
    


## 问题16：[找到冠军 I](https://leetcode.cn/problems/find-champion-i/description/)
### 题目
给你一个下标从0开始、大小为n * n的二维布尔矩阵grid 。如果 grid[i][j] == 1，那么i队比j队强；否则j队比i队强。(i != j)。请返回出最强的队。要求复杂度为O(n)

### 分析
- 我们可以遍历每一个grid[i], 如果其中的比分只有一个为0(即只有自己grid[i][i]=0), 说明这个i比其他人都要强，但这种算法复杂度为O(n^2)
- 我们可以使用打擂台的方式, 假设最强队为ans=0, 我们从1开始遍历, 遍历grid[i][ans] = 1, 说明i比目前的ans强, 我们更新ans=i, 否则继续考察grid[i+1][ans], 即ans一定是前[0, i] 最强的选手。

In [ ]:
def process(grid):
    ans = 0
    for index, group in enumerate(grid):
        print(index)
        if group[ans] == 1:  # 比较index和ans谁比较强
            ans = index    # 更新当前最强者
    return ans

process([[0,1],[0,0]])

## 问题17：[找到冠军 II](https://leetcode.cn/problems/find-champion-ii/description/)
### 题目
一场比赛中共有 n 支队伍，按从 0 到  n - 1 编号, 给你一个整数 n 和一个下标从 0 开始、长度为 m 的二维整数数组 edges 表示这个有向无环图，其中 edges[i] = [u, v] 表示图中存在一条从 u 队到 v 队的有向边。表示队伍u强于队伍v。如果这场比赛存在**唯一**一个冠军, 则返回将会成为冠军的队伍。否则，返回 -1 (可能存在a队与b队没有进行过比赛)

### 分析
- 使用淘汰赛机制, 开始时ans = [0, n-1], 遍历每条边当出现[u, v]时, 在ans中去掉v, 如果最后剩下的ans只有一个则为最强者, 如果有多个则返回-1

In [ ]:
def process(n, edges):
    ans = set(range(n))
    for u, v in edges:
        if v in ans:
            ans.remove(v)
    
    return ans.pop() if len(ans) == 1 else -1


## 问题18：[尽量减少恶意软件的传播](https://leetcode.cn/problems/minimize-malware-spread/description/)
### 题目
由 n 个节点组成的网络，用 n × n 个邻接矩阵图 graph 表示。在节点网络中，当 graph[i][j] = 1 时，表示节点i能够直接连接到另一个节点 j(graph[i][j]=graph[j][i], 且graph[i][i]=1).一些节点 initial 最初被恶意软件感染。只要两个节点直接连接，且其中至少一个节点受到恶意软件的感染，那么两个节点都将被恶意软件感染。这种恶意软件的传播将继续，直到没有更多的节点可以被这种方式感染。

请从 initial 中移除一个节点能够使得最终的受感染节点数最少, 返回该节点。(如果有多个节点满足条件，就返回索引最小的节点)。

### 分析
- 找到受感染节点中的最大联通块。(类似于统计树中合法路径数目, 遍历每个节点, 如果当前节点为质数, 则将当前连通块的节点个数加到结果中, 最终得到的结果就是“不包含质数的连通块”集合)
- 遍历每个节点, 如果当前节点为污染节点, 则将当前连通块的节点个数进行记录. 使用一个size数组来记录每个非污染连通块的大小, 例如size[1]表示包含1这个节点的污染连通块的大小。
- 而如果连通块内至少有两个节点被感染，无论移除哪个节点，仍然会导致连通块被感染.所以我们要找的是**只包含一个**被感染节点的连通块，并且这个连通块越大越好


In [ ]:

def process(graph, initial):
    n = len(graph)
    initial = set(initial)

    # 得到邻接图, G[i]表示与i节点有边的其他节点
    Graph = [[] for _ in range(n+1)]  
    for i in range(n-1):
        for j in range(i+1, n):
            if graph[i][j] == 1:
                Graph[i].append(j)
                Graph[j].append(i)
    print("邻接图: ", Graph)

    # 计算以node为根节点的连通块大小. 如果该连通块中又出现了一个受污染节点, 那么我们将标识置为false(意味着当前连通块中还存在至少一个污染节点)
    def dfs(node, father):
        nonlocal size, flag, vis
        size += 1
        vis[node] = True           # 不能根据flag提前返回, 需要将所有的这个block块中节点标记为 已经访问
        if node in initial:        # 如果当前节点为污染节点, 则说明当前块中已经有2个污染节点了
            flag = False
        for child in Graph[node]:  # 遍历node的所有邻居
            if vis[child] or child == father:  # 如果子节点已经被访问过, 或遍历到父节点了, 则跳过
                continue
            if child in initial:   # 如果为污染节点则标识为false
                flag = False
            dfs(child, father)
    
    vis = [False] * n   # 记录访问情况, false为未访问
    ans_index = -1      # 需要删除的节点index
    max_nodes = 0       # 受感染最大联通块中的节点数量
    for x in initial:   # 遍历每个污染节点
        if vis[x]:      # 如果该节点已经被访问过, 说明该节点已经被计算过
            continue
        size = 1
        flag = True
        # 统计x(受污染的节点)的所有邻居
        for y in Graph[x]:
            if not vis[y]:       # y没有被计算过
                # 拿到y为根的联通块
                dfs(y, x)
        
        if size > max_nodes and flag==True:  # 更新只包含一个受污染节点的最大联通块数量
            print("更新", x)
            max_nodes = size
            ans_index = x
    # 如果ans_index = -1则说明删除任意一个节点都不能降低受感染节点总数量
    return min(initial) if ans_index == -1 else ans_index

graph = [[1,1,0], [1,1,0], [0,0,1]]
initial = [0,1,2]
process(graph, initial)


## 问题19：[从双倍数组中还原原数组](https://leetcode.cn/problems/find-original-array-from-doubled-array/description/)
### 题目
一个整数数组original可以转变成一个双倍数组changed，转变方式为将original中每个元素值乘以2加入数组中，然后将所有元素随机打乱。现在该你一个双倍后的数组changed，请你返回原始数组original。如果无法找到原始数组则返回[]


### 分析
- 将元素进行排序, 遍历每个元素, 如果其在need_nums中则元素次数-1, 如果不在，则将其2倍的数值放入need_nums中(或者数量+1)，并将其放入到ans中，遍历完成后如果need_nums中存在元素则返回ans，否则返回[]

In [ ]:
from collections import defaultdict
def process(changed):
    changed_sort = sorted(changed)
    need_nums = defaultdict(int)
    ans = []
    print(changed_sort)
    for i in changed_sort:
        if i in need_nums:
            need_nums[i] -= 1
            if need_nums[i] == 0:
                del need_nums[i]
            print(need_nums)
        else:
            need_nums[i*2] += 1
            ans.append(i)

    return ans if len(need_nums) == 0 else []

changed = [2,1,2,4,2,4]
process(changed)

## 问题20：[]()
### 题目


### 分析